<a href="https://colab.research.google.com/github/dajoeng/KDT_team4_FinalPJT/blob/main/02_modeling/%EB%A0%88%EC%8B%9C%ED%94%BC_%EC%B6%94%EC%B2%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive


# 레시피 추천 시스템

In [2]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec

In [3]:
coupang = pd.read_csv('coupang_filtering.csv')
coupang.drop('Unnamed: 0', axis=1, inplace=True)

naver_df = pd.read_csv('2022-12-29edited_navertrend.csv')
naver_df.drop('Unnamed: 0', axis=1, inplace=True)

In [79]:
ckg = pd.read_csv('ckg_drop.csv')
ckg.drop('Unnamed: 0', axis=1, inplace=True)
ckg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97378 entries, 0 to 97377
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   레시피일련번호   97378 non-null  int64 
 1   레시피제목     97378 non-null  object
 2   요리명       95875 non-null  object
 3   조회수       97378 non-null  int64 
 4   추천수       97378 non-null  int64 
 5   스크랩수      97378 non-null  int64 
 6   요리방법별명    97378 non-null  object
 7   요리상황별명    96911 non-null  object
 8   요리재료별명    97378 non-null  object
 9   요리종류별명    97378 non-null  object
 10  요리소개      97018 non-null  object
 11  요리재료내용    97378 non-null  object
 12  요리재료_전처리  97374 non-null  object
dtypes: int64(4), object(9)
memory usage: 9.7+ MB


## 만개의 레시피에서 조회수 기반 스크랩수 가중치 계산

In [77]:
def weighted_rating(v,m,R,C):
    '''
    Calculate the weighted rating
    
    Args:
    v -> average rating for each item (float)
    m -> minimum votes required to be classified as popular (float)
    R -> average rating for the item (pd.Series)
    C -> average rating for the whole dataset (pd.Series)
    # ( (각 리뷰수 / (각 리뷰수+최소리뷰수)) * 각 별점 ) + ( (최소리뷰수/(각 리뷰수+최소리뷰수)) * 평균별점 )
    
    Returns:
    pd.Series
    '''
    return ( (v / (v + m)) * R) + ( (m / (v + m)) * C )

In [80]:
# calcuate input parameters
C = np.mean(ckg['스크랩수'])
m = np.percentile(ckg['조회수'], 70)
vote_count = ckg[ckg['조회수'] >= m]
R = ckg['스크랩수']
v = ckg['조회수']

ckg['weighted_clipping'] = weighted_rating(v,m,R,C)
ckg.sort_values('weighted_clipping')

,레시피일련번호,레시피제목,요리명,조회수,추천수,스크랩수,요리방법별명,요리상황별명,요리재료별명,요리종류별명,요리소개,요리재료내용,요리재료_전처리,weighted_clipping
47555,6881612,깍두기/깍두기 만드는법~ 김치중에서도 제일 쉬운 깍두기...,깍두기,105092,0,26,무침,일상,채소류,김치/젓갈/장류,김치중에서도 제일 쉬운 깍두기 혹시 어렵게만 생각하고 계시지는 않으시나요?? 김치...,[재료] 무 1k 굵은소금 1주먹,무 굵은소금,66.559005
68539,6912997,#마파두부소스 #마파두부만들기 #마파두부덮밥만들기 #청양고추 팍팍 넣고 만든 마파두부!!,마파두부,122170,0,34,볶음,손님접대,돼지고기,메인반찬,마파두부소스를 활용한 초스피드 마파두부덮밥입니다. 마라소스를 구입하면서 마파두부소...,[재료] 마파 두부 소스 80g| 두부 2모| 돼지고기다짐육 500g| 모둠채소(냉...,"마파 두부, 두부, 돼지고기다짐육, 모둠채소, 썬파, 다진마늘, 고추기름, 치킨스톡...",68.659458
95617,6950069,#콩나물국#시원한콩나물국#담백한콩나물국#디포리육수콩나물국,콩나물국,119788,0,35,끓이기,일상,채소류,국/탕,기본적으로 아무맛도 나지않은 그러나 간을 맞춰서 육수를 우려내 넣고 끓이면 시원하...,[재료] 콩나물 200g| 디포리 or멸치 10마리| 양파 1/4개| 홍고추 1개|...,"콩나물, 디포리 멸치, 양파, 홍고추, 대파, 두부, 다진마늘, 소금, 새우젓, 물",70.179069
68886,6913487,(- 전복요리 : 전복손질법+전복죽끓이는방법/전복내장죽으로 몸보신-!,전복내장죽,79479,0,20,기타,영양식,해물류,밥/죽/떡,안녕하십니까 쇼핑중입니다. 오늘은 선물받은 전복이 있어서 전복요리에 도전해보려고해...,[전복손질] 칫솔| 수저 [전복볶기] 참기름 또는 식용유 1T [전복끓이기] 밥...,"칫솔, 수저, 참기름 또는, 밥, 물",72.611519
80904,6930295,콩자반 만드는법 도시락 반찬 메뉴 부드러운 콩자반,콩자반,81684,0,24,끓이기,일상,콩/견과류,메인반찬,부드럽게 콩자반 만들어 볼께요.,[재료] 콩1컵| 콩삶은 물600ml| 간장3T| 올리고당 2T| 통깨,"콩, 콩삶은, 간장, 올리고당, 통깨",74.828467
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49357,6884469,단짠단짠 너무맛있는별미 간장쪽갈비,간장쪽갈비,1302963,31,73959,찜,손님접대,돼지고기,메인반찬,주말 특별한날의 남녀노소 + 아이들까지도 맛있게 먹을 수 있는 쪽갈비 달달하면서...,[재료] 등갈비 1kg| 된장 1T| 통마늘 5개 [ 양념] 간장 8T| 설탕 4T...,"등갈비, 된장, 통마늘, 간장, 설탕, 물엿, 매실액, 맛술, 다진마늘, 참기름, ...",73211.767830
46204,6879533,생생정보통 잡채 황금레시피 이거였네,잡채,1880362,42,77293,볶음,일상,기타,메인반찬,신랑 생일날 생생정보통 잡채 레시피로 만들어봤어요 간단하면서도 맛있어서 배불리 잘먹...,[재료] 당면 1줌반| 당근 약간| 파프리카 약간| 목이버섯 1줌| 양파 1/2개|...,"당면, 당근, 파프리카, 목이버섯, 양파, 부추, 잡채용 돼지고기, 다진마늘, 진간...",76750.062348
55537,6894096,너무 간단한데 맛있어서 놀라는 백종원 분식점 떡볶이 황금 레시피,떡볶이,1717002,20,86382,볶음,초스피드,가공식품류,밥/죽/떡,분식점에서 파는 맛 나는 백주부님 떡볶이!만드는 방법은 너무 간단한데 맛있어서 놀랜...,[재료] 떡볶이떡 2컵| 물 2컵| 대파 1/2대| 통깨 약간 [양념] 고추장 2T...,"떡볶이떡, 물, 대파, 통깨, 고추장, 고추가루, 간장, 설탕",85717.582541
49465,6884636,부대찌개 양념 만드는법 홀릭되는 맛임,부대찌개,2164414,22,88748,끓이기,일상,가공식품류,찌개,?밖에서 파는 맛처럼 맛있는 부대찌개 양념 만드는 법! 고추장을 넣으면 텁텁해지면서...,[재료] 김치 1/4밥공기(1줌)| 비엔나 20개| 스팸 1개| 두부 1/3모| 떡...,"김치, 비엔나, 스팸, 두부, 떡, 대파 센티, 홍고추, 청양고추 큰거, 사골곰...",88205.558555


# word2ve 기반

In [83]:
coupang.iloc[100:101, :]

,카테고리명,상품id,data-item-id,data-vendor-item-id,상품,상품명,정가,할인율,판매가,100g당_가격,별점,리뷰수,품절여부,구성정보,구성정보_전처리,상세카테고리
100,1,6285333706,12925369588,80189918728,온기원 사골 왕만두 떡국 키트 (냉동),"온기원 사골 왕만두 떡국 키트 (냉동), 1315g, 1개",10900,0,10900,(100g당 829원),4.5,297,1,"만두, 소스(육수), 떡, 부추, 대파, 알가열제품(지단)으로 구성되어 있습니다.","만두,소스,육수,떡,부추,대파,지단",국


In [93]:
product = coupang.iloc[100:101, :].상품.values
product_list = np.array2string(product).split(' ')

category = coupang.iloc[100:101, :].상세카테고리.values

recipe_df = pd.DataFrame()
for word in product_list:
  recipe = ckg.loc[ckg['요리명']==word]
  recipe_df = pd.concat([recipe_df, recipe])
#ckg[ckg.요리명 == '왕만두']

In [94]:
recipe_df

,레시피일련번호,레시피제목,요리명,조회수,추천수,스크랩수,요리방법별명,요리상황별명,요리재료별명,요리종류별명,요리소개,요리재료내용,요리재료_전처리,weighted_clipping
16655,6814508,영양가득 사골 끓이기,사골,6004,4,33,끓이기,영양식,소고기,국/탕,뼈에 좋은 사골 끌이기!!,[재료] 사골| 잡뼈| 양지| 파| 마늘 [양념] 소금| 후추,"사골, 잡뼈, 양지, 파, 마늘, 소금, 후추",276.177478
90647,6943470,사골끓이는법/아파트에서 할거 다하기 2편/Beef bone soup/깔끔한 사골 진...,사골,1656,0,37,끓이기,일상,기타,국/탕,사골끓이는게 막막하게 느껴지지만 막상 해보면 시간 투자하는게 힘들어서 그렇지 의외로...,[재료] 사골 1.5kg| 소고기사태 500g| 대파 1/2개| 소금 약간,"사골, 소고기사태, 대파, 소금",346.990102
93928,6947756,[에코즈 레시피]사골 끓이는법,사골,609,0,7,끓이기,일상,소고기,국/탕,추울수록 국물요리가 많이 생각이 나고 요즘일수록 몸을 보해주는 음식이 많이 더오르죠...,[재료] 사골뼈 1kg| 잡뼈 1kg| 물 1냄비가득,"사골뼈, 잡뼈, 물",368.902366
79877,6928879,반죽에서 속재료까지~자화자찬하게 만드는 왕만두&찐빵,왕만두,3718,0,61,찜,일상,기타,면/만두,저는 특별히 좋아하는 음식이 별로 없는 입맛이죠 그렇다고 입맛이 없는 것도 아니에...,[재료] 밀가루 500g| 물 200g| 소금 2/3T| 설탕 3t| 계란 1개| ...,"밀가루, 물, 소금, 설탕, 계란, 드라이이스트, 올리브오일, 다진돼지고기, 부추,...",314.803785
93829,6947641,왕만두 만들기,왕만두,1859,0,18,찜,일상,돼지고기,메인반찬,익은김치가 맛있어 라면사리 넣어서 간단하게 왕만두 만들기 라면을 넣어더니 더 맛있대요,[재료] 돼지고기 300g| 김치 4/1쪽| 두부 300g| 숙주 200g| 청양고...,"돼지고기, 김치, 두부, 숙주, 청양고추, 부추, 달걀, 간장, 설탕, 매실청, 후...",340.538237
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95423,6949796,새해 복 많이 받으세요.오색떡국떡으로 신년떡국 끓이는법,떡국,875,0,11,끓이기,명절,쌀,국/탕,세해첫날 떡국은 드셨나요. 오색떡국떡과 얼려 두었던 사골국과 달걀지단을 넣어 초간단...,[재료] 떡국떡 400g| 물만두 100g| 사골국물 4컵| 달걀지단 조금| 소고기...,"떡국떡, 물만두, 사골국물, 달걀지단, 소고기, 표고버섯, 대파, 다진마늘, 후추,...",362.389727
95495,6949917,떡국떡으로 두부꾸미 福 떡국,떡국,954,0,10,끓이기,명절,쌀,국/탕,새해새날 떡국 맛있게 드셨나요? 다음엔 특별하게 두부꾸미도 만들어서 떡국에 올려보세...,[재료] 떡국떡 1 kg| 소고기 채끝 등심 400 g| 파 1 단| 부침용 두부 ...,"떡국떡, 소고기 채끝, 파, 부침용 두부, 유부주머니, 단밤, 간장, 다진마늘, 소...",360.384995
95671,6950144,떡국 맛있게 끓이는법 / 떡국떡 쌀떡,떡국,8298,0,156,끓이기,명절,소고기,국/탕,설날이 아니더라도 김치만 있어도 맛있어서 지인들에게 자주 해주는 요리입니다.,[재료] 떡국떡 400g| 한우 국거리 50g| 들기름 1ts| 채썬 대파 1큰술|...,"떡국떡, 한우, 들기름, 채썬 대파, 다진마늘, 후추, 김가루, 계란, 국간장, 굵은소금",297.537374
95956,6950507,떡국 끓이는법 간단하고 맛있게,떡국,1911,0,15,끓이기,초스피드,쌀,국/탕,맛있게 간단한 레시피 별미로 먹어도 맛있는 떡국,[재료] 떡국떡 1컵반정도| 멸치육수 400ml| 계란 1개| 김(고명김)| 대파|...,"떡국떡, 멸치육수, 계란, 김, 대파, 국간장",339.065398


In [71]:
word = [words for words in recipe_df.요리재료_전처리]
embedding_model = Word2Vec(sentences=word, size=100, window=5, min_count=1,  sg=1, batch_words=1000)

In [65]:
# 단어 벡터 평균구하기
def vectors(embedding):
    word_embedding=[]
    dec2vec=None
    count=0
    for word in embedding:
        try : 
            # word에 해당하는 단어를 워드투백에서 찾아 리스트에 붙힘
            # 100차원으로 형성됨
            word_embedding.append(embedding_model.wv[word]) 
            count+=1
        except : continue
    # print('='*30,"word_embedding1","="*30)
    # print(word_embedding)
    
    word_embedding2=np.array(word_embedding)
    # print('='*30,"word_embedding2","="*30)
    # print(word_embedding2)
    
    v=word_embedding2.sum(axis=0)  
    # print(v)
    
    if type(v)!=np.ndarray:
        return np.zeros(100)
    
    return(v/count)

In [66]:
wordvec=[vectors(x) for x in word]

In [67]:
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances, manhattan_distances

# 1. 코사인유사도
w2v_sim=cosine_similarity(wordvec,wordvec)
# 2. 유클리디언 유사도
w2v_euc=euclidean_distances(wordvec,wordvec)
# 3. 맨하탄 유사도 
w2v_man=manhattan_distances(wordvec,wordvec)

In [116]:
#최종 알고리즘
def sim_mealkit(mealkit,sim,top_n):
    
    # 찾고자 하는 옷선택
    mealkits=coupang[coupang.구성정보_전처리==mealkit]

    # 찾고자 하는 옷의 인덱스 값만 추출하여 새로운 변수에 저장
    mealkits_index=mealkits.index.values
    
    # 코사인유사도를 구한 행렬을 역순으로 정렬 -> 유사도가 높은 순의 인덱스로 정렬됨 
    sim_sorted_ind=sim.argsort()[:,::-1]
    
    # 내가 추출하려는 옷의 인덱스의 행번째추출, 이중리스트이므로 [0][0]
    # -> 아이템이름을 추출 -> 내가 원하는 옷의 이름과 같지 않다면
    if recipe_df.iloc[sim_sorted_ind[mealkits.index][0][0]].요리재료_전처리 != mealkit:
        
        # 내가 추출하려는 옷의 인덱스 행번째의 array를 리스트로 변경, 변수명 l로 저장
        l=sim_sorted_ind[mealkits.index][0].tolist()
        # 내가 추출하려는 옷의 인덱스 값 자체를 리스트로 변경한 후 값으로 추출
        v=mealkits.index.values.tolist()[0]
        
        # 리스트 l 에서 내가 추출하려고 한 옷의 값을 제거
        # -> 내가 추출하고자 하는 옷의 인덱스 번호가 제거된 리스트 생성
        l.remove(mealkits.index.values.tolist()[0])
        
        # 리스트l의 array형태로 맞춰주기
        sim_sorted_ind2=np.array(l)
        
        # 자기 자신을 제외하고 유사도가 높은 인덱스를 top_n개수 만큼 추출
        # 내가 추출하고자 하는 인덱스의 값이 제거된 상태이므로 0값부터 원하는 개수만큼 출력
        similar_indexes=sim_sorted_ind2[0:(top_n)]
        a=  recipe_df.iloc[similar_indexes].sort_values(by='weighted_clipping',ascending=False)
    
    else :
        # sim_indexes는 이중리스트 상태이므로 이중리스트를 해제한 후 인덱스를 이용해 해당 내용 추출
        similar_indexes=sim_sorted_ind[mealkits_index,1:(top_n+1)]
        a= recipe_df.iloc[similar_indexes[0]].sort_values(by='weighted_clipping',ascending=False)
    
    return a

In [113]:
str_list = coupang.iloc[100:101, :].구성정보_전처리.values.tolist()
ingredient = ' '.join(s for s in str_list)
ingredient

'만두,소스,육수,떡,부추,대파,지단'

In [119]:
sim_mealkit_df = sim_mealkit(ingredient, w2v_sim, 5)
sim_mealkit_df 

,레시피일련번호,레시피제목,요리명,조회수,추천수,스크랩수,요리방법별명,요리상황별명,요리재료별명,요리종류별명,요리소개,요리재료내용,요리재료_전처리,weighted_clipping
84821,6935696,#잔치국수용분말 #시원한떡국만들기 #만두국만들기 #잔치국수용분말을 이용해서 손쉽게 ...,떡국,1161,0,6,끓이기,초스피드,가공식품류,국/탕,잔치국수용분말 시원한떡국만들기 만두국만들기 오늘은 주말!! 조금은 편하게 그리고 ...,[재료] 떡국떡 2컵| 만두 7개| 당근 1/4개| 양파 1/2개| 잔치국수용분말 ...,"떡국떡, 만두, 당근, 양파, 잔치국수용분말, 후추, 물, 참기름, 다진마늘, 썬파...",355.073660
40512,6870215,1인분 떡국 후딱 만들기~,떡국,18945,16,294,끓이기,일상,쌀,국/탕,혼자 밥먹는 시간이 많다보니 밥 차리긴 귀찮고 라면도 지겹고.. 그럴땐 냉동실 한켠...,[재료] 떡국떡 2줌| 육수용멸치(육수가있다면육수)| 다진마늘 0.5T| 국간장 1...,"떡국떡, 육수용멸치, 다진마늘, 국간장, 소금, 후추, 대파, 달걀",331.809255
61780,6903117,사골 육수로 끓인 떡국,떡국,15305,0,192,끓이기,일상,쌀,국/탕,냉동실에 있던 사골육수를 녹혀 맛난 떡국 끓여 먹었어요~,[재료] 떡국떡 5인분| 사골육수| 다진마늘| 국간장 1큰술| 소금| 대파| 후춧가루,"떡국떡, 사골육수, 다진마늘, 국간장, 소금, 대파, 후춧가루",282.165525
76589,6924367,떡국 끓이기...,떡국,7244,0,40,끓이기,일상,쌀,국/탕,2020년 새해에 먹은 떡국이랍니다.,[재료] 떡국떡 500g정도| 사골육수 1리터정도| 물 500ml정도| 소고기 국거...,"떡국떡, 사골육수, 물, 소고기, 대파, 다진마늘, 소금, 다시다, 후춧가루",264.008843
53411,6890856,떡국 만드는 법 : 손쉬운 떡국 레시피 : 밤에 배고플때 제격,떡국,10215,0,63,끓이기,야식,쌀,국/탕,배가고파서 떡국 만들어 봤어요. 손 쉬운 떡국 만드는 법 공유 돌김이 떡국속에 스며...,[재료] 떡국용떡 1인분양| 간장 1.5T| 후추 약간| 들깻가루 조금| 대파 조금...,"떡국용떡, 간장, 후추, 들깻가루, 대파, 계란, 소금, 다진마늘, 다시마, 멸치,...",245.751326


In [125]:
print(ingredient)
recipe_ingredient = []
for items in sim_mealkit_df.요리재료_전처리:
  items = items.split(', ')
  for item in items:
    if item in str_list:
        items.remove(item)
  recipe_ingredient.append(items)
recipe_ingredient

만두,소스,육수,떡,부추,대파,지단


[['떡국떡',
  '만두',
  '당근',
  '양파',
  '잔치국수용분말',
  '후추',
  '물',
  '참기름',
  '다진마늘',
  '썬파',
  '청양고추',
  '달걀',
  '오이나물',
  '통깨',
  '김가루'],
 ['떡국떡', '육수용멸치', '다진마늘', '국간장', '소금', '후추', '대파', '달걀'],
 ['떡국떡', '사골육수', '다진마늘', '국간장', '소금', '대파', '후춧가루'],
 ['떡국떡', '사골육수', '물', '소고기', '대파', '다진마늘', '소금', '다시다', '후춧가루'],
 ['떡국용떡',
  '간장',
  '후추',
  '들깻가루',
  '대파',
  '계란',
  '소금',
  '다진마늘',
  '다시마',
  '멸치',
  '말린갈치',
  '양파',
  '돌김',
  '파슬리가루']]

In [121]:
euc_mealkit_df = sim_mealkit(ingredient, w2v_euc, 5)
euc_mealkit_df 

,레시피일련번호,레시피제목,요리명,조회수,추천수,스크랩수,요리방법별명,요리상황별명,요리재료별명,요리종류별명,요리소개,요리재료내용,요리재료_전처리,weighted_clipping
93928,6947756,[에코즈 레시피]사골 끓이는법,사골,609,0,7,끓이기,일상,소고기,국/탕,추울수록 국물요리가 많이 생각이 나고 요즘일수록 몸을 보해주는 음식이 많이 더오르죠...,[재료] 사골뼈 1kg| 잡뼈 1kg| 물 1냄비가득,"사골뼈, 잡뼈, 물",368.902366
76577,6924349,멸치 다싯물에 끓인 시원한 떡국,떡국,2469,0,9,끓이기,일상,가공식품류,국/탕,사골국 보단 다싯물에 끓인 떡국을 더 좋아하는 우리 식구들의 취향에 맞춰서 시원하게...,[재료] 떡국떡 국그릇으로1그릇| 시판만두 1/2봉| 계란 1개| 김 1장 [양념]...,"떡국떡 국그릇으로, 시판만두, 계란, 김, 다싯물, 소금, 참기름",326.712671
25496,6842863,떡국,떡국,6105,1,9,끓이기,일상,쌀,국/탕,소고기와 고명4가지를 올린 떡국이랍니다.,[재료] 소고기 300그램| 표고버섯 고명| 팽이버섯 고명| 달걀 지단 고명| 송선...,"소고기, 표고버섯, 팽이버섯, 달걀 지단, 송선미의, 진간장, 청정원 남해안",267.392384
76353,6924029,초간단 떡국 끓이기,떡국,11878,0,58,끓이기,일상,곡류,국/탕,떡국은 설날에만 먹는 것? 아니쥬~ ㅎ 떡국은 언제먹어도 좋은데요 ? 오늘은...,[재료] 백미떡국떡 1/2줌| 감귤 떡국떡 1.5줌 [양념] 멸치다싯포 1개| 대파...,"백미떡국떡, 감귤 떡국떡, 멸치다싯포, 대파, 달걀, 국간장",231.361282
25352,6842651,설날 떡국 쉽고 맛있게 끓이는법,떡국,15232,8,61,끓이기,명절,쌀,국/탕,설날 대표음식 떡국입니다~,[재료] 떡국떡 2줌| 멸치다시마 육수 500cc| 달걀 1개| 불고기양념소고기 약...,"떡국떡, 멸치다시마, 달걀, 불고기양념소고기, 대파푸른부분, 국간장, 소금",212.629149


In [126]:
print(ingredient)
recipe_ingredient = []
for items in euc_mealkit_df.요리재료_전처리:
  items = items.split(', ')
  for item in items:
    if item in str_list:
        items.remove(item)
  recipe_ingredient.append(items)
recipe_ingredient

만두,소스,육수,떡,부추,대파,지단


[['사골뼈', '잡뼈', '물'],
 ['떡국떡 국그릇으로', '시판만두', '계란', '김', '다싯물', '소금', '참기름'],
 ['소고기', '표고버섯', '팽이버섯', '달걀 지단', '송선미의', '진간장', '청정원 남해안'],
 ['백미떡국떡', '감귤 떡국떡', '멸치다싯포', '대파', '달걀', '국간장'],
 ['떡국떡', '멸치다시마', '달걀', '불고기양념소고기', '대파푸른부분', '국간장', '소금']]

In [122]:
man_mealkit_df = sim_mealkit(ingredient, w2v_man, 5)
man_mealkit_df 

,레시피일련번호,레시피제목,요리명,조회수,추천수,스크랩수,요리방법별명,요리상황별명,요리재료별명,요리종류별명,요리소개,요리재료내용,요리재료_전처리,weighted_clipping
93928,6947756,[에코즈 레시피]사골 끓이는법,사골,609,0,7,끓이기,일상,소고기,국/탕,추울수록 국물요리가 많이 생각이 나고 요즘일수록 몸을 보해주는 음식이 많이 더오르죠...,[재료] 사골뼈 1kg| 잡뼈 1kg| 물 1냄비가득,"사골뼈, 잡뼈, 물",368.902366
76577,6924349,멸치 다싯물에 끓인 시원한 떡국,떡국,2469,0,9,끓이기,일상,가공식품류,국/탕,사골국 보단 다싯물에 끓인 떡국을 더 좋아하는 우리 식구들의 취향에 맞춰서 시원하게...,[재료] 떡국떡 국그릇으로1그릇| 시판만두 1/2봉| 계란 1개| 김 1장 [양념]...,"떡국떡 국그릇으로, 시판만두, 계란, 김, 다싯물, 소금, 참기름",326.712671
25496,6842863,떡국,떡국,6105,1,9,끓이기,일상,쌀,국/탕,소고기와 고명4가지를 올린 떡국이랍니다.,[재료] 소고기 300그램| 표고버섯 고명| 팽이버섯 고명| 달걀 지단 고명| 송선...,"소고기, 표고버섯, 팽이버섯, 달걀 지단, 송선미의, 진간장, 청정원 남해안",267.392384
76353,6924029,초간단 떡국 끓이기,떡국,11878,0,58,끓이기,일상,곡류,국/탕,떡국은 설날에만 먹는 것? 아니쥬~ ㅎ 떡국은 언제먹어도 좋은데요 ? 오늘은...,[재료] 백미떡국떡 1/2줌| 감귤 떡국떡 1.5줌 [양념] 멸치다싯포 1개| 대파...,"백미떡국떡, 감귤 떡국떡, 멸치다싯포, 대파, 달걀, 국간장",231.361282
25352,6842651,설날 떡국 쉽고 맛있게 끓이는법,떡국,15232,8,61,끓이기,명절,쌀,국/탕,설날 대표음식 떡국입니다~,[재료] 떡국떡 2줌| 멸치다시마 육수 500cc| 달걀 1개| 불고기양념소고기 약...,"떡국떡, 멸치다시마, 달걀, 불고기양념소고기, 대파푸른부분, 국간장, 소금",212.629149


In [128]:
print(ingredient)
recipe_ingredient = []
for items in man_mealkit_df.요리재료_전처리:
  items = items.split(', ')
  for item in items:
    if item in str_list:
        items.remove(item)
  recipe_ingredient.append(items)
recipe_ingredient

만두,소스,육수,떡,부추,대파,지단


[['사골뼈', '잡뼈', '물'],
 ['떡국떡 국그릇으로', '시판만두', '계란', '김', '다싯물', '소금', '참기름'],
 ['소고기', '표고버섯', '팽이버섯', '달걀 지단', '송선미의', '진간장', '청정원 남해안'],
 ['백미떡국떡', '감귤 떡국떡', '멸치다싯포', '대파', '달걀', '국간장'],
 ['떡국떡', '멸치다시마', '달걀', '불고기양념소고기', '대파푸른부분', '국간장', '소금']]

# tfidf 기반